# Registery Query MVP Demo Notebook 
> A tutorial of RRAP Registry Query using Jupyter notebooks.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter, registry]
<!-- - image: images/chart-preview.png -->

## About

This page captures simple demonstrations of some of capabilities of RRAP to Registry Activities, Entities and Actors.  If you select the 'Binder' button above, a Jupyter Notebook will be lauched that you can use to further explore these examples.  You can change parameters and see results to further expand your understand of RRAP Registry.

With `RRAP-IS Registry` system can be queried to find all 'Activities', 'Entities' and 'Actors' associated with particular 'Outputs', 'Inputs', or 'Reef Interventions'

Please read documenation ### to understand terms used in this notebook and the future systems



## Import general dependencies

In [165]:
import requests
import os
import sys
import json
import json2tree
from bs4 import BeautifulSoup
from IPython.display import IFrame, display, HTML, JSON, Markdown
from mdsisclienttools.auth.TokenManager import DeviceFlowManager

import warnings
warnings.filterwarnings(action='once')

### Define global variables, base urls
Note the use structure of the base url, pointing to test servers.  If you wish to test your code first, please use the test servers (listed below).  Note that these may not retain data and could be down at the time of testing.

Remove `testing` from the url to use the production urls.  Please be careful using production urls, filling the system with test data will require considerable effort to cleanup.

In [166]:

data_api = "https://data-api.testing.rrap-is.com"
registry_api = "https://registry-api.testing.rrap-is.com"
prov_api = "https://prov-api.testing.rrap-is.com"
auth_server = "https://auth.dev.rrap-is.com/auth/realms/rrap"
# garbage = "https://frogs.are.green"
base_urls = {'data_api': data_api, 'registry_api': registry_api, 'prov_api': prov_api, 'auth_server': auth_server}#, 'garbage': garbage}
display('Checking base urls')
# [display(Testing':',value) for key, value in base_urls.items()]
for key, url in base_urls.items():
    try:
        print(f'Testing - {url}', end="")
        r = requests.get(url)
        r.raise_for_status()
        print(f' - Passed')
    except requests.exceptions.HTTPError as err:
        print(f' - Fail')
        raise SystemExit(err)
    except requests.exceptions.RequestException as e:
        # catastrophic error. bail.
        print(f' - Fail')
        raise SystemExit(e)

'Checking base urls'

Testing - https://data-api.testing.rrap-is.com - Passed
Testing - https://registry-api.testing.rrap-is.com - Passed
Testing - https://prov-api.testing.rrap-is.com - Passed
Testing - https://auth.dev.rrap-is.com/auth/realms/rrap - Passed


### Setup tokens using device authorisation flow against keycloak server

This could result in a browser window being opened if you don't have valid tokens cached in local storage.

In [167]:
# this caches the tokens
local_token_storage = ".tokens.json"

token_manager = DeviceFlowManager(
    stage="TEST",
    keycloak_endpoint=auth_server,
    local_storage_location=local_token_storage
)

Attempting to generate authorisation tokens.

Looking for existing tokens in local storage.

Validating found tokens

Found tokens valid, using.



## Convert json output to html for display

In [168]:
def convert_json_to_html(json_input):
    out_file = open("example.json", "w")
    json.dump(json_input, out_file)
    out_file.close()
    !json2tree -j "example.json" -o "output.html" -t 1
    with open("output.html") as f:
        cleaned_list = remove_unwanted(f)
        list_elements = extract_list(cleaned_list)
        wrapped_list = wrap_details(cleaned_list)
    
    with open("clean.html", 'w') as fp:
        fp.write(wrapped_list)
        
    with open('clean.html', 'r') as fin:
        data = fin.read().splitlines(True)
    with open('clean.html', 'w') as fout:
        fout.writelines(data[1:])

In [169]:
def remove_unwanted(data):
    soup = BeautifulSoup(data, features="html.parser")
    soup = remove_class(soup)
    span_tag = soup.span.extract()
    nav_tag = soup.nav.extract()
    head_tag = soup.head.extract()
    return soup.prettify()

In [170]:
def extract_list(data):
    soup = BeautifulSoup(data, features="html.parser")
    list_tag = soup.ul.extract()
    return list_tag.prettify()

In [171]:
def wrap_details(data):
    soup = BeautifulSoup(data)

    ul_tag = soup.find("ul")
    div_tag = soup.new_tag("div")
    div_tag['style'] = "width: 800px; height: 400px; overflow-y: scroll;"
    ul_tag.wrap(div_tag)
    new_tag = soup.new_tag("details")
    div_tag.wrap(new_tag)
    
    tag = soup.new_tag("summary")
    tag.string = "Results"
    soup.div.insert_after(tag)

    return soup.prettify()

In [172]:
def remove_class(f):
    with open("output.html") as f:
        soup = BeautifulSoup(f, features="html.parser")

        for tag in soup():
            for attribute in ["class"]: # You can also add id,style,etc in the list
                del tag[attribute]

    return soup

# Demonstrations

## Dataset Registry APIs
### List all datasets

In [ ]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/items/list-all-datasets"
endpoint = data_api + postfix

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.get(endpoint, auth=auth())

convert_json_to_html(response.json())

HTML('clean.html')

## Model Registry APIs
### List all registered models

In [174]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/entity/model/list"
endpoint = registry_api + postfix

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.get(endpoint, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')

### Fetch registered model
From the above output copy the handle id and use it to fetch 

In [175]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/entity/model/fetch"
params = {"id" : "10378.1/1687616"}
endpoint = registry_api + postfix

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.get(endpoint, params=params, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')

### Register a model

In [176]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/entity/model/create"
payload = {
    "display_name" : "RRAP-model (repo commit id - 1234)",
    "name" : "RRAP-model-commit-1234",
    "url" : "https://bitbucket.csiro.au/projects/MAE/repos/pybrat/commits/824fc954c7e85c8b4b94044fc22cdbf83e365735"
}
endpoint = registry_api + postfix 

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.post(endpoint, json=payload, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')

### Delete a registered model

In [177]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/entity/model/delete"
params = { 'id': '10378.1/1687609' }
endpoint = registry_api + postfix

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.delete(endpoint, params=params, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')

Token validation failed due to error: Signature has expired.
Refreshing using refresh token



## Configuration File Registry APIs
### Create a Configuration File registry

In [178]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/entity/config_file/create"
payload = { 
    "display_name" : "Demo Notebook Configuration File",
    "url" : "https://www.fake.model.com/demo_config_file"
}
endpoint = registry_api + postfix 

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.post(endpoint, json=payload, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')

### Fetch Configuration File

In [179]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/entity/config_file/fetch"
params = "?id=10378.1/1687612"
endpoint = registry_api + postfix + params

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.get(endpoint, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')

## Parameter File Registry APIs
### Create Parameter File

In [180]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/entity/parameter_file/create"
payload = { 
    "display_name" : "Demo Notebook Parameter File",
    "url" : "https://www.fake.model.com/demo_pramater_file"
}
endpoint = registry_api + postfix 

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.post(endpoint, json=payload, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')

### Fetch Parameter File

In [181]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/entity/parameter_file/fetch"
params = { 'id': '10378.1/1687613' }
endpoint = registry_api + postfix 

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.get(endpoint, params=params, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')

## Agent Registry APIs
### Create a Agent

In [182]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/agent/person/create"
payload = { 
    "display_name" : "Reef Modeller for eReef",
    "first_name" : "John",
    "last_name" : "Doe",
    "email" : "John.Doe@uni-of.au",
    "orcid" : "https://orcid.org/0000-0000-000-0000"
}
endpoint = registry_api + postfix 

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.post(endpoint, json=payload, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')

### List all Agents

In [183]:


# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/agent/person/list"
endpoint = registry_api + postfix

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.get(endpoint, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')

## Organistation Registry APIs
### Create a Organisation

In [184]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/agent/organisation/create"
payload = { 
    "display_name" : "Reef Research",
    "name" : "Reef Research"
}
endpoint = registry_api + postfix 

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.post(endpoint, json=payload, auth=auth())

convert_json_to_html(response.json())
HTML('clean.html')